In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import sklearn


In [11]:
df = pd.read_csv (r'C:\BOOTCAMP_DATA_SCIENCE\Team Challenge\Team Challenge - Despliegue de Modelo en Producción\car_price_dataset.csv')

df.sample(10)

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
2365,Mercedes,GLA,2020,2.3,Electric,Manual,124582,5,3,11908
5734,BMW,X5,2009,1.8,Petrol,Semi-Automatic,281531,4,1,2969
5399,Chevrolet,Impala,2003,4.8,Petrol,Semi-Automatic,43123,2,4,8937
9434,Volkswagen,Golf,2000,1.4,Diesel,Manual,7764,5,4,5344
8504,BMW,X5,2020,4.2,Petrol,Manual,177156,5,1,10756
9555,Chevrolet,Malibu,2000,4.8,Petrol,Automatic,238974,5,5,5620
7497,Volkswagen,Golf,2022,2.3,Petrol,Semi-Automatic,95978,3,1,11080
1205,Audi,A4,2021,4.6,Petrol,Automatic,118860,3,2,14122
281,Toyota,Camry,2000,1.8,Electric,Automatic,271405,4,4,3971
8571,Chevrolet,Malibu,2001,2.9,Electric,Semi-Automatic,57004,5,1,8159


In [9]:
df.describe()

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000


In [12]:
# Dividir features y target
X = df.drop('Price', axis=1)  # Ajusta esto si tu columna objetivo tiene otro nombre
y = df['Price']                # Ajusta esto si tu columna objetivo tiene otro nombre

# Identificar columnas categóricas y numéricas
categorica_col = ['Brand', 'Model', 'Fuel_Type', 'Transmission']
numerica_col = ['Year', 'Engine_Size', 'Mileage', 'Doors', 'Owner_Count']

# Crear preprocesadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerica_col),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorica_col)
    ])

# Crear pipeline con preprocesador y modelo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=20, random_state=42))
])

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model_pipeline.fit(X_train, y_train)

# Evaluar el modelo
score = model_pipeline.score(X_test, y_test)
print(f"R² Score: {score}")

# Guardar el modelo entrenado
with open('modelo_coche.pkl', 'wb') as file:
    pickle.dump(model_pipeline, file)

print("Modelo guardado correctamente como 'modelo_coche.pkl'")

R² Score: 0.9852118285659666
Modelo guardado correctamente como 'modelo_coche.pkl'


In [ ]:
# Crear la aplicación FastAPI
app = FastAPI(title="API de Predicción de Precios de Coches")

# Cargar el modelo guardado
try:
    with open('modelo_coche.pkl', 'rb') as file:
        model = pickle.load(file)
except FileNotFoundError:
    model = None
    print("¡ADVERTENCIA! Modelo no encontrado. La API no funcionará correctamente.")

# Definir la estructura de datos de entrada
class CarFeatures(BaseModel):
    Brand: str
    Model: str
    Year: int
    Engine_Size: float
    Fuel_Type: str
    Transmission: str
    Mileage: int
    Doors: int
    Owner_Count: int

# Endpoint raíz - Página de inicio
@app.get("/")
async def root():
    return {
        "mensaje": "¡Bienvenido a la API de predicción de precios de coches!",
        "endpoints": {
            "/": "Esta página de inicio que muestra información de la API",
            "/predict": "Endpoint para predecir el precio de un coche (POST)",
            "/docs": "Documentación automática de la API"
            # El tercer endpoint estará comentado y se habilitará durante la presentación
        },
        "ejemplo_predict": {
            "Brand": "Toyota",
            "Model": "Corolla",
            "Year": 2015,
            "Engine_Size": 1.8,
            "Fuel_Type": "Petrol",
            "Transmission": "Manual",
            "Mileage": 50000,
            "Doors": 5,
            "Owner_Count": 2
        }
    }

# Endpoint de predicción
@app.post("/predict")
async def predict_price(car: CarFeatures):
    if model is None:
        raise HTTPException(status_code=500, detail="Modelo no cargado correctamente")
    
    # Convertir los datos de entrada en un DataFrame
    car_df = pd.DataFrame([car.dict()])
    
    # Hacer la predicción
    try:
        prediction = model.predict(car_df)
        # Redondear a 2 decimales y convertir a valor flotante
        price = float(round(prediction[0], 2))
        
        return {
            "coche": car.dict(),
            "precio_predicho": price,
            "mensaje": f"El precio estimado del coche es: {price} euros"
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error en la predicción: {str(e)}")



# Para ejecutar localmente
if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)

INFO:     Will watch for changes in these directories: ['c:\\BOOTCAMP_DATA_SCIENCE\\Team Challenge\\Team Challenge - Despliegue de Modelo en Producción']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [3912] using StatReload


In [5]:
print(sklearn.__version__)


1.5.2
